In [3]:
def path_cost(path : list, weights : dict, cycle=True) -> int:
    """Computes the cost of the given path

    Args:
        path (list): tour taken\n
        weights (dict): key -> edge : value -> weight.\n
        cycle (bool, optional): [description]. Defaults to True.\n

    Returns:
        int: final cost
    """
    cost = 0
    for i in range(len(path) - 1):
        try:
            if (path[i], path[i + 1]) in weights.keys():
                cost += weights[(path[i],path[i+1])]
            else:
                cost += weights[(path[i + 1],path[i])]
        except:
            if cycle:
                if (path[i], path[0]) in weights.keys():
                    cost += weights[(path[i], path[0])]
                else:
                    cost += weights[(path[0], path[i])]
                pass
    return cost


In [4]:
import math
def compute_distance(a, b):
    a = a['pos']
    b = b['pos']
    return math.sqrt(math.pow(a[0]-b[0],2) + math.pow(a[1]-b[1],2))


In [5]:
from graph import MyGraph
import torch
import os

def create_graph_fittizio(
     name : str,
     dir : str,
     lb : int = 0,
     ub : int = 100,
     n_nodes : int = 50,
     #normalized : bool = False,
     count : int = 0
    ):
    G = nx.Graph()
    V = set([v + 1 for v in range(n_nodes)])
    
    E = set()
    for combination in combinations(V, 2):
        E.add(combination)
    coords = []
    to_normalize_location = []
    max_coord = 0
    for i in range(1,n_nodes + 1):
        location = (random.randint(lb,ub), random.randint(lb,ub))
        if location[0] > max_coord:
            max_coord = location[0]
        elif location[1] > max_coord:
            max_coord = location[1]
        to_normalize_location.append(location)
    #print(to_normalize_location)
    for i, location in enumerate(to_normalize_location):
        normalized_location = (location[0]/max_coord, location[1]/max_coord)
        #print(normalized_location)
        coords.append(normalized_location)
        G.add_node(i+1, pos = normalized_location)
    coords = torch.Tensor(coords)
    for edge in E:
        w = compute_distance(G.nodes[edge[0]], G.nodes[edge[1]])
        #print(edge[0], edge[1], w)
        G.add_edge(edge[0], edge[1], weight = w)
    
    tsp = nx.approximation.traveling_salesman_problem
    path = tsp(G, cycle=True)
    if len(path) > 51:
        return count
    path = np.array(path)
    w_dic = {}
    for i in range(1, 51):
        for j in range(1,51):
            if i != j:
                w_dic[(i,j)] = compute_distance(G.nodes[i], G.nodes[j])
    #print(w_dic)
    cost = path_cost(path, weights = w_dic)
    #print(path, cost)
    

    #TODO salvare i grafi creati
    g = MyGraph(name + '.mio', nodes = G.nodes(), coords = coords,weights = w_dic, sub_opt = path, sub_opt_cost= cost)
    #print(len(path))
    path_to_save = os.path.join(dir, name)
    torch.save(g, path_to_save)
    return count + 1

/opt/anaconda3/envs/Adm_project/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import networkx as nx
import numpy as np
import random
from itertools import combinations

def get_weight(a, b):
    a = a['pos']
    b = b['pos']
    return int(np.sqrt((a[0]-b[0])**2 + (b[1]-a[1])**2))



In [7]:
from utility import create_dir
name_dir = 'generated_test'
create_dir(name_dir)
count = 0
while count < 10000:
    count = create_graph_fittizio(f'{name_dir}_' + str(count) + '.mio', name_dir, ub = 1000, count = count)

Path created correctly
[ 1 25 22 13 50 39 14 24 48 38 28  7 46 36  8  3 42 16 20 26 30 27 45 47
 23  9  2 11 21 34 32 12 17  5 43 33 40 35 44 29 49 19 15  6 10 18 41 37
  4 31  1] 5.7559053843063115
[ 1 18 29 35 30 36  4  6 32 10 48 37 24 44 14  5 22  8 33 23 50  7 34 16
 21 13 43 28 20 42 46 25  3 27 26 17 12 45  9 41  2 31 19 39 38 15 11 40
 49 47  1] 6.55504695543852
[ 1 46 36  7 40 34 28 20 29  2 50 37 21  4 27 30 31  5  6 17 39 26 23 49
 32 10 18 48 22 45 38 19 11 35 16  8  9 43  3 25 41 13 47 24 42 14 15 44
 12 33  1] 6.404307482121547
[ 1 37  4 23  2 47 27  6 20 19  7  5 44 42 26 49 48 16 36  8 38 12 31 17
 15 28 35 14 34  9 10 40 50 41  3 39 29 24 11 25 32 45 33 30 22 21 46 18
 13 43  1] 5.88788206953417
[ 1 23 18 24 45  6 12 44 10 30  5 42  9 27 28 33 36 47  3 16 20  2 34  8
 49 35 25 26 17 41 19 48 40 37 11 46 50  4 13  7 43 39 14 29 22 31 32 15
 21 38  1] 6.3705153428435075
[ 1 14 36  8 19 20 12 30 29  7 23 18 17 47 39 35 16 42 45 37 15 46 28 34
  9 31  6  4  3 49 38 21 33 2

In [8]:
for i in range(10000):
    x = torch.load(f'{name_dir}/{name_dir}_{i}.mio')
    if (len(x.sub_opt)) > 51:
        print(x.sub_opt_cost)
        print(x.coords)


In [9]:
from utility import create_dir
name_dir = 'generated_eval'
create_dir(name_dir)
count = 0
while count < 5000:
    count = create_graph_fittizio(f'{name_dir}_' + str(count) + '.mio', name_dir, ub = 1000, count = count)

Path created correctly
[ 1 18  9 21 36 13 46 25 16  7 33 34 30 40 29  4 15 50 17 19 26 41 44 43
  3 35 42 24 39 47 22 32 14  8 20 23 28 37 31 27  2 49 11 12 45 10  5 48
 38  6  1] 5.441587630984728
[ 1  2 17 35 39  7 21 43  6 23 29 22 34 27 50 44 30 41 15 25 40 31 36 38
 16 10 42 19 24  9 49 46 11 47 32  8 18 12  5  4 28 20 33 26  3 37 13 45
 48 14  1] 5.8189394492657724
[ 1  7 24 21 43 49 25 46 47 29 30  9  5  8 35 48 16 27 44 18 50 23 42 45
 31 13 15 38 37  6 26 12  3  2 10 41 39 17 32 11 28 19 33 22 14 20  4 40
 34 36  1] 6.856547450554646
[ 1 10 32 39  4 50 41 14 29  5 21 20 33 26 44 47 38 23 46  7 13  2 48  6
 24 42 37 36 18  3 27 11  9 30 19 28 25 49 16 31  8 34 43 45 15 40 35 12
 22 17  1] 6.477656722016265
[ 1 19 16 26 30 39  5 34 22 32 42 33 13 31 29  8 10 35 46 37  2 45 15 47
 41 40  4 17 21 43 44 36 24 50 48 12 14 49  3 23 25 18  9  6 11 20 28  7
 27 38  1] 5.838717697773516
[ 1 43 42 11  3 50 12  9 41 45 19 14  4 48 40 13 30  7 26 21 44 35 34 38
  5 36 25 20 29 17 27 18 28 

In [ ]:
for i in range(5000):
    x = torch.load(f'{name_dir}/{name_dir}_{i}.mio')
    if (len(x.sub_opt)) != 51 or (x.sub_opt_cost > 8):
        print(i)
        print(x.coords.shape)
        print(x.coords)
        print(x.nodes)
